# 🔄 Comparación: OpenAI vs Google Gemini

**Notebook de Referencia**: Comparación práctica entre OpenAI GPT y Google Gemini para tareas de ingeniería de datos.

## Objetivos
- Entender diferencias entre proveedores de LLMs
- Comparar precios, latencia y calidad de respuestas
- Aprender a implementar fallback entre providers
- Elegir el modelo adecuado según el caso de uso

## 1. Setup de Ambos Proveedores

In [ ]:
import os
import time
from openai import OpenAI
import google.generativeai as genai

# Configurar APIs
client_openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

print('✅ OpenAI y Gemini configurados')

## 2. Funciones de Consulta Unificadas

In [ ]:
def query_openai(prompt: str, model: str = 'gpt-3.5-turbo') -> dict:
    """Consulta a OpenAI con timing."""
    start = time.time()
    
    response = client_openai.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0.7
    )
    
    elapsed = time.time() - start
    
    return {
        'provider': 'OpenAI',
        'model': model,
        'response': response.choices[0].message.content,
        'latency_ms': round(elapsed * 1000, 2),
        'tokens': response.usage.total_tokens if hasattr(response, 'usage') else None
    }

def query_gemini(prompt: str, model: str = 'gemini-1.5-flash') -> dict:
    """Consulta a Gemini con timing."""
    start = time.time()
    
    model_gemini = genai.GenerativeModel(model)
    response = model_gemini.generate_content(prompt)
    
    elapsed = time.time() - start
    
    return {
        'provider': 'Google Gemini',
        'model': model,
        'response': response.text,
        'latency_ms': round(elapsed * 1000, 2),
        'tokens': None  # Gemini no retorna token count directamente
    }

def query_both(prompt: str) -> None:
    """Consulta ambos proveedores y compara."""
    print(f'\n❓ Prompt: {prompt}\n')
    
    # OpenAI
    result_openai = query_openai(prompt)
    print(f'🟢 {result_openai["provider"]} ({result_openai["model"]}) - {result_openai["latency_ms"]}ms')
    print(f'   {result_openai["response"][:200]}...')
    
    # Gemini
    result_gemini = query_gemini(prompt)
    print(f'\n🔵 {result_gemini["provider"]} ({result_gemini["model"]}) - {result_gemini["latency_ms"]}ms')
    print(f'   {result_gemini["response"][:200]}...')

## 3. Comparación: Tareas Simples

In [ ]:
# Pregunta conceptual
query_both('¿Qué es un Data Lakehouse en 2 frases?')

## 4. Comparación: Generación de SQL

In [ ]:
sql_prompt = '''
Genera una query SQL que:
- Tabla: ventas (fecha DATE, producto TEXT, cantidad INT, total DECIMAL)
- Calcula ventas totales por mes del último año
- Ordena de mayor a menor
'''

query_both(sql_prompt)

## 5. Comparación: Generación de Código

In [ ]:
code_prompt = '''
Genera una función Python que:
1. Lee un CSV con pandas
2. Filtra filas donde columna "status" == "active"
3. Agrupa por "category" y suma "revenue"
4. Retorna un DataFrame ordenado por revenue descendente

Solo el código, sin explicaciones.
'''

query_both(code_prompt)

## 6. Tabla Comparativa de Precios

In [ ]:
import pandas as pd

pricing_data = [
    {
        'Provider': 'OpenAI',
        'Model': 'gpt-3.5-turbo',
        'Input ($/1M tokens)': 0.50,
        'Output ($/1M tokens)': 1.50,
        'Use Case': 'Desarrollo, prototipos'
    },
    {
        'Provider': 'OpenAI',
        'Model': 'gpt-4',
        'Input ($/1M tokens)': 10.00,
        'Output ($/1M tokens)': 30.00,
        'Use Case': 'Producción, tareas complejas'
    },
    {
        'Provider': 'Google',
        'Model': 'gemini-1.5-flash',
        'Input ($/1M tokens)': 0.075,
        'Output ($/1M tokens)': 0.30,
        'Use Case': 'Desarrollo, alto volumen'
    },
    {
        'Provider': 'Google',
        'Model': 'gemini-1.5-pro',
        'Input ($/1M tokens)': 1.25,
        'Output ($/1M tokens)': 5.00,
        'Use Case': 'Producción, equilibrio costo-calidad'
    },
    {
        'Provider': 'Google',
        'Model': 'gemini-1.5-pro (128K+)',
        'Input ($/1M tokens)': 2.50,
        'Output ($/1M tokens)': 10.00,
        'Use Case': 'Contextos largos (documentación)'
    }
]

df_pricing = pd.DataFrame(pricing_data)
print('\n💰 Comparación de Precios (Oct 2025)\n')
print(df_pricing.to_string(index=False))

## 7. Estrategia de Fallback

In [ ]:
def query_with_fallback(prompt: str, prefer_gemini: bool = True) -> dict:
    """Consulta con fallback automático."""
    providers = [query_gemini, query_openai] if prefer_gemini else [query_openai, query_gemini]
    
    for provider_func in providers:
        try:
            result = provider_func(prompt)
            print(f'✅ Respuesta de {result["provider"]}')
            return result
        except Exception as e:
            print(f'⚠️ Error en {provider_func.__name__}: {e}')
            continue
    
    raise Exception('Todos los proveedores fallaron')

# Test
resultado = query_with_fallback('¿Qué es Apache Kafka?')
print(resultado['response'][:200])

## 8. Recomendaciones por Caso de Uso

### ✅ Usa **Gemini 1.5 Flash** para:
- Desarrollo y testing
- Alto volumen de queries (APIs públicas)
- Tareas simples (clasificación, extracción)
- Presupuesto limitado
- **Ahorro**: 85% vs GPT-3.5, 99% vs GPT-4

### ✅ Usa **GPT-3.5 Turbo** para:
- Prototipos rápidos con ecosistema OpenAI
- Compatibilidad con herramientas existentes
- Tareas que requieren función calling

### ✅ Usa **Gemini 1.5 Pro** para:
- Producción con buen balance costo-calidad
- Contextos largos (hasta 1M tokens)
- RAG sobre documentación extensa
- **Ahorro**: 87% vs GPT-4

### ✅ Usa **GPT-4** para:
- Tareas críticas que requieren máxima precisión
- Razonamiento complejo
- Code generation de alta calidad
- Cuando el costo no es limitante

## 9. Benchmark de Latencia

In [ ]:
def benchmark_latency(prompts: list, iterations: int = 3) -> pd.DataFrame:
    """Benchmark de latencia."""
    results = []
    
    for prompt in prompts:
        for _ in range(iterations):
            # OpenAI
            result_openai = query_openai(prompt)
            results.append({
                'provider': 'OpenAI GPT-3.5',
                'latency_ms': result_openai['latency_ms']
            })
            
            # Gemini
            result_gemini = query_gemini(prompt)
            results.append({
                'provider': 'Gemini Flash',
                'latency_ms': result_gemini['latency_ms']
            })
            
            time.sleep(1)  # Rate limiting
    
    df_results = pd.DataFrame(results)
    return df_results.groupby('provider')['latency_ms'].agg(['mean', 'min', 'max', 'std'])

# Ejecutar benchmark
test_prompts = [
    'Explica ETL en 1 frase',
    'SELECT * FROM users WHERE age > 18',
    '¿Qué es ACID?'
]

print('\n⏱️ Benchmark de Latencia (3 iteraciones)\n')
benchmark_results = benchmark_latency(test_prompts)
print(benchmark_results)

## 10. Configuración Recomendada

In [ ]:
# Configuración sugerida en .env
recommended_config = '''
# Desarrollo
DEFAULT_LLM_PROVIDER=gemini
DEFAULT_GEMINI_MODEL=gemini-1.5-flash
FALLBACK_PROVIDER=openai
FALLBACK_MODEL=gpt-3.5-turbo

# Producción
PROD_LLM_PROVIDER=gemini
PROD_GEMINI_MODEL=gemini-1.5-pro
PROD_FALLBACK_PROVIDER=openai
PROD_FALLBACK_MODEL=gpt-4

# Límites
MAX_TOKENS=4000
TIMEOUT_SECONDS=30
MAX_RETRIES=3
'''

print(recommended_config)

## Conclusiones

### 🎯 Resumen

**Ganador en Costo**: Gemini Flash (85-99% más barato)
**Ganador en Calidad**: GPT-4 (ligeramente superior en tareas complejas)
**Mejor Balance**: Gemini Pro (calidad cercana a GPT-4 a 87% menos costo)
**Mejor Latencia**: Variable, depende de la región y carga

### 💡 Estrategia Recomendada

1. **Desarrollo**: Gemini Flash con fallback a GPT-3.5
2. **Producción no-crítica**: Gemini Pro
3. **Producción crítica**: GPT-4 con caché agresivo
4. **Siempre**: Implementa fallback entre proveedores
5. **Monitoreo**: Trackea costos, latencia y calidad de respuestas

### 🔧 Próximos Pasos

- Implementa un router inteligente que elija el modelo según la tarea
- Usa modelos pequeños (Flash/3.5) para clasificación
- Usa modelos grandes (GPT-4) solo para generación compleja
- Considera modelos locales (Ollama) para datos sensibles

---

## 🧭 Navegación

**← Anterior:** [← README del Curso](../../README.md)

**Siguiente →:** [🤖 Fundamentos de LLMs y Prompting →](01_fundamentos_llms_prompting.ipynb)

**📚 Índice de Nivel GenAI:**
- [🔄 Comparación: OpenAI vs Google Gemini](00_comparacion_openai_gemini.ipynb) ← 🔵 Estás aquí
- [🤖 Fundamentos de LLMs y Prompting](01_fundamentos_llms_prompting.ipynb)
- [📊 Text-to-SQL: Generación de Consultas SQL desde Lenguaje Natural](02_generacion_sql_nl2sql.ipynb)
- [🔧 Generación Automática de Código ETL con LLMs](03_generacion_codigo_etl.ipynb)
- [📚 RAG: Documentación Técnica con LLMs](04_rag_documentacion_datos.ipynb)
- [🔍 Embeddings y Similitud en Datos](05_embeddings_similitud_datos.ipynb)
- [🤖 Agentes Autónomos para Automatización](06_agentes_automatizacion.ipynb)
- [✅ Validación de Datos con LLMs](07_calidad_validacion_llm.ipynb)
- [🎲 Generación de Datos Sintéticos con LLMs](08_sintesis_aumento_datos.ipynb)
- [🚀 Proyecto Integrador 1: Chatbot de Consulta de Datos con RAG](09_proyecto_integrador_1.ipynb)
- [🏗️ Proyecto Integrador 2: Plataforma Self-Service con GenAI](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
